Import necessary modules

In [44]:
import deepchem as dc
from deepchem.feat.rdkit_grid_featurizer import RdkitGridFeaturizer
import glob
import os
import re
from tqdm import *
import csv
import pickle
import multiprocessing as mp
import math
import numpy as np

Get all the mol and pdb files

In [2]:
dataset_path = "/home/mhassan/pdbbind_data_set/refined_set/"
ligand_files = sorted(glob.glob(os.path.join(dataset_path, '*/*.sdf')))
receptor_files = sorted(glob.glob(os.path.join(dataset_path, '*/*protein.pdb')))
print('Receptor files count: ', len(receptor_files), 'Ligand files count: ', len(ligand_files))

Receptor files count:  4154 Ligand files count:  4154


Define the featurizer

Available voxel feature in deepchem : 'ecfp', 'splif', 'sybyl', 'salt_bridge', 'charge', 'hbond', 'pi_stack, 'cation_pi' <br>
(from https://deepchem.io/docs/_modules/deepchem/feat/rdkit_grid_featurizer.html) <br>

nb_rotations=0,<br>
               feature_types=None, [voxel_combined includes all available voxel features] <br>
               ecfp_degree=2,<br>
               ecfp_power=3,<br>
               splif_power=3,<br>
               box_width=16.0,<br>
               voxel_width=1.0,<br>
               flatten=False,<br>
               verbose=True,<br>
               sanitize=False<br>

In [22]:
grid_featurizer = RdkitGridFeaturizer(
    feature_types='voxel_combined',
    voxel_feature_types=['ecfp', 'splif', 'salt_bridge', 'hbond'], 
    parallel=True, 
    flatten=True,
    verbose=False)

Read experimental values

In [4]:
exp_values = {}
with open('../trainDLSCORE/exp_values_from_pbd_bind.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader, None)
    for row in reader:
        data = {}
        data['affinity_data'] = row[4]
        data['pkd'] = float(row[5])
        exp_values[row[1]] = data

Method for feature extraction

In [5]:
def extract_features(feature_dict, mols, pdbs, n):
    text = "Progressor #{:>2}".format(n)
    with tqdm_notebook(total=len(mols), desc=text, position=n) as pbar:
        for mol, pdb in zip(mols, pdbs):
            f = re.findall('\w\w\w\w/', mol)
            pdb_id = f[len(f)-1].strip('/')

            try:
                features = grid_featurizer.featurize_complexes(mol_files=[mol], protein_pdbs=[pdb])
            except:
                pbar.update()
                pass
            feature_dict[pdb_id] = {'features':features, 'pkd':exp_values[pdb_id]['pkd']}
            pbar.update()

Yield chunks of receptor and ligand files

In [6]:
def chunks(list1, list2, n):
    assert len(list1)==len(list2)
    """Yield successive n-sized chunks from a list."""
    for i in range(0, len(list1), n):
        yield list1[i:i + n], list2[i:i + n]

Define the number of process to run and the chunk sizes

In [7]:
process_count = mp.cpu_count() # Number of process to be created
chunk_size = math.ceil(len(receptor_files)/process_count)
#chunk_size = 2 # For testing purpose
key_chunks = [(j, k) for j, k in chunks(ligand_files, receptor_files, chunk_size)]

Define the job manager and the combined feature dictionary

In [8]:
mgr = mp.Manager()
feature_dict = mgr.dict()

In [9]:
jobs = [mp.Process(target=extract_features, args=(feature_dict, k[0], k[1], n)) for n, k in enumerate(key_chunks)]

In [10]:
print('Total jobs count: ', len(jobs))

Total jobs count:  56


Start all the jobs and accumulate the results

In [11]:
for j in jobs:
    j.start()
for j in jobs:
    j.join()

Featurizing 0 / 1
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.002 s
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.003 s
Featurizing 0 / 1
TIMING: Writing protein took 0.002 s
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing ligand took 0.000 s
Featurizing 0 / 1
Featurizing 0 / 1
TIMING: Writing protein took 0.002 s
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.016 s
TIMING: Writing protein took 0.001 s
TIMING: Writing ligand took 0.000 s
Featurizing 0 / 1
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.001 s
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.005 s
Featurizing 0 / 1
TIMING: Writing protein took 0.003 s
TIMING: Writing ligand took 0.000 s
Featurizing 0 / 1
TIMING: Writing protein took 0.003 s
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing li

/home/mhassan/.conda/envs/deepchem-env/lib/python3.5/site-packages/simtk/openmm/app/internal/pdbstructure.py:442: UserWarning: WARNING: two consecutive residues with same number (HETATM 3185  O   HOH   112     -15.460   3.310  17.954  1.00 60.56           O  , HETATM 3184  OXT CAS   112     -12.813   2.091  17.927  1.00 66.00           O  )
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (atom, self._current_residue.atoms[-1]))


TIMING: Loading protein coordinates took 14.120 s
TIMING: Loading protein coordinates took 21.124 s
TIMING: Loading protein coordinates took 28.679 s
TIMING: Loading protein coordinates took 29.310 s
TIMING: Loading protein coordinates took 32.544 s
TIMING: Loading protein coordinates took 39.709 s
TIMING: Loading protein coordinates took 51.313 s
TIMING: Loading ligand coordinates took 39.664 s
TIMING: Centroid processing took 0.003 s
TIMING: ecfp voxel computataion took 0.303 s
TIMING: splif voxel computataion took 0.419 s
TIMING: hbond voxel computataion took 0.298 s
TIMING: salt_bridge voxel computataion took 0.064 s


/home/mhassan/.conda/envs/deepchem-env/lib/python3.5/site-packages/deepchem/feat/rdkit_grid_featurizer.py:1118: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dtype=np.int8)


Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.003 s
TIMING: Writing protein took 0.002 s
TIMING: Loading protein coordinates took 55.933 s
TIMING: Loading protein coordinates took 57.083 s
TIMING: Loading protein coordinates took 57.964 s
TIMING: Loading protein coordinates took 58.657 s
TIMING: Loading protein coordinates took 58.670 s
TIMING: Loading protein coordinates took 58.944 s
TIMING: Loading ligand coordinates took 38.618 s
TIMING: Centroid processing took 0.000 s
TIMING: ecfp voxel computataion took 0.363 s
TIMING: splif voxel computataion took 0.574 s
TIMING: hbond voxel computataion took 0.521 s
TIMING: salt_bridge voxel computataion took 0.143 s


...

TIMING: Loading ligand coordinates took 6.752 s
TIMING: Centroid processing took 0.000 s
TIMING: salt_bridge voxel computataion took 0.134 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.006 s
TIMING: splif voxel computataion took 1.189 s
TIMING: hbond voxel computataion took 1.022 s
TIMING: salt_bridge voxel computataion took 0.217 s
Completed ecfp tensor
TIMING: splif voxel computataion took 1.556 s
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.012 s
TIMING: ecfp voxel computataion took 1.308 s
TIMING: hbond voxel computataion took 0.886 s
TIMING: salt_bridge voxel computataion took 0.099 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.00

TIMING: salt_bridge voxel computataion took 0.047 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.001 s
TIMING: Loading protein coordinates took 29.605 s
TIMING: Loading ligand coordinates took 1.279 s
TIMING: Centroid processing took 0.000 s
TIMING: ecfp voxel computataion took 1.239 s
TIMING: Loading ligand coordinates took 2.298 s
TIMING: Centroid processing took 0.001 s
TIMING: splif voxel computataion took 1.358 s
TIMING: Loading protein coordinates took 14.528 s
TIMING: hbond voxel computataion took 1.455 s
TIMING: salt_bridge voxel computataion took 0.172 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.006 s
TIMING: ecfp voxel computataion took 2.731 s
TIMING: Loading ligand coordinates took 1.721 s
TIMING: Ce

Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.002 s
TIMING: splif voxel computataion took 2.896 s
TIMING: splif voxel computataion took 1.401 s
TIMING: Loading protein coordinates took 23.653 s
TIMING: ecfp voxel computataion took 2.424 s
TIMING: Loading ligand coordinates took 0.793 s
TIMING: Centroid processing took 0.001 s
TIMING: splif voxel computataion took 2.783 s
TIMING: hbond voxel computataion took 1.451 s
TIMING: salt_bridge voxel computataion took 0.245 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.006 s
TIMING: hbond voxel computataion took 2.556 s
TIMING: salt_bridge voxel computataion took 0.305 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000

TIMING: splif voxel computataion took 0.651 s
TIMING: Loading protein coordinates took 14.350 s
TIMING: hbond voxel computataion took 0.678 s
TIMING: salt_bridge voxel computataion took 0.070 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.002 s
TIMING: Loading ligand coordinates took 1.909 s
TIMING: Centroid processing took 0.000 s
TIMING: Loading protein coordinates took 8.240 s
TIMING: ecfp voxel computataion took 1.280 s
TIMING: Loading protein coordinates took 10.177 s
TIMING: splif voxel computataion took 1.452 s
TIMING: Loading ligand coordinates took 1.995 s
TIMING: Centroid processing took 0.000 s
TIMING: Loading ligand coordinates took 1.825 s
TIMING: Centroid processing took 0.001 s
TIMING: Loading protein coordinates took 17.740 s
TIMING: hbond voxel computataion took 1.164 s
TIMING: ecfp voxel computataion took 0.242 s
TIMING: ecfp voxel 

Completed salt_bridge tensor.
TIMING: Loading protein coordinates took 7.175 s
TIMING: Loading ligand coordinates took 1.207 s
TIMING: Centroid processing took 0.000 s
TIMING: ecfp voxel computataion took 0.297 s
TIMING: splif voxel computataion took 0.392 s
TIMING: hbond voxel computataion took 0.341 s
TIMING: salt_bridge voxel computataion took 0.091 s
Completed ecfp tensor
Completed splif tensor
Completed hbond tensor
Completed salt_bridge tensor.
Featurizing 0 / 1
TIMING: Writing ligand took 0.000 s
TIMING: Writing protein took 0.001 s
TIMING: Loading protein coordinates took 21.575 s
TIMING: Loading protein coordinates took 9.788 s
TIMING: Loading ligand coordinates took 1.952 s
TIMING: Centroid processing took 0.001 s
TIMING: Loading ligand coordinates took 1.816 s
TIMING: Centroid processing took 0.000 s
TIMING: ecfp voxel computataion took 1.263 s
TIMING: ecfp voxel computataion took 0.619 s
TIMING: splif voxel computataion took 0.790 s
TIMING: Loading protein coordinates took 

In [38]:
print('Size of the feature dictionary: ', len(feature_dict))

Size of the feature dictionary:  4079


Save the dictionary as a pickle file

In [21]:
with open('feature_dict.pickle', 'wb') as f:
    pickle.dump(dict(feature_dict), f)

Get the shapes

In [49]:
sample = next(iter(feature_dict.values()))
print(sample.keys())
sample_feature = sample['features']
print(sample_feature.shape)

dict_keys(['features', 'pkd'])
(1, 16, 16, 16, 36)


Define variables

In [42]:
dict_len = len(feature_dict)
cube_height = sample_feature.shape[1]
cube_width = sample_feature.shape[2]
cube_depth = sample_feature.shape[3]
nb_features = sample_feature.shape[4]

Define numpy arrays to save the x and y data

In [51]:
data_x = np.zeros([dict_len, cube_height, cube_width, cube_depth, nb_features]).astype(np.float32)
print(data_x.shape)
data_y = np.zeros(dict_len).astype(np.float32)
print(data_y.shape)

(4079, 16, 16, 16, 36)
(4079,)


Save the features and labels in numpy arrays

In [53]:
for i, k in enumerate(feature_dict.keys()):
    data_x[i] = feature_dict[k]['features']
    data_y[i] = feature_dict[k]['pkd']

Mean normalize the data

In [56]:
mean = np.mean(data_x, axis=0)
std = np.std(data_x, axis=0)

In [58]:
data_x_norm = (data_x - mean)/(std + 0.00001)

Save the arrays in pickle files

In [63]:
with open('data_x_norm.pickle', 'wb') as f1, open('data_y.pickle', 'wb') as f2:
    pickle.dump(data_x_norm, f1)
    pickle.dump(data_y, f2)